<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/15_coin_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import time

# CoinGecko API erişim bilgileri
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def get_coingecko_data(symbols, vs_currency='usd'):
    """CoinGecko API üzerinden birden fazla kripto para verisini tek istekte çeker."""
    symbol_str = ','.join(symbols)
    params = {
        'vs_currency': vs_currency,
        'ids': symbol_str.lower(),
        'order': 'market_cap_desc',
        'per_page': len(symbols),
        'page': 1,
        'sparkline': False
    }

    try:
        response = requests.get(COINGECKO_URL, params=params, headers=HEADERS, timeout=15)
        response.raise_for_status()
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            df = df[['id', 'current_price', 'high_24h', 'low_24h']]
            df.rename(columns={'id': 'Coin', 'current_price': 'Close', 'high_24h': 'High', 'low_24h': 'Low'}, inplace=True)
            df['Timestamp'] = pd.to_datetime(time.time(), unit='s')
            return df
    except requests.exceptions.RequestException as e:
        print(f"API Hatası: {e}")

    return pd.DataFrame()

def calculate_devisso_trend(df, ma_period=20):
    """Basit hareketli ortalama (SMA) hesaplayarak trend sinyali üretir."""
    if df.empty:
        return df

    df['MA'] = df['Close'].rolling(ma_period, min_periods=1).mean()
    df['Signal'] = np.where(df['Close'] > df['MA'], 'BUY', 'SELL')
    return df

def scan_coins(coin_list):
    """Belirtilen coin listesindeki trend sinyallerini tek API isteğiyle tarar."""
    data = get_coingecko_data(coin_list)
    if not data.empty:
        return calculate_devisso_trend(data)
    return pd.DataFrame()

def save_to_excel(df, filename="crypto_signals.xlsx"):
    """Veriyi Excel dosyasına kaydeder."""
    try:
        df.to_excel(filename, index=False)
        print(f"📊 Excel dosyası kaydedildi: {filename}")
    except Exception as e:
        print(f"Excel kaydetme hatası: {e}")

# Çalıştırma
coin_list = [
    'bitcoin', 'ethereum', 'binancecoin', 'cardano', 'solana',
    'xrp', 'dogecoin', 'polkadot', 'litecoin', 'chainlink',
    'uniswap', 'monero', 'avalanche-2', 'shiba-inu', 'stellar'
]

while True:
    print(f"\n--- Tarama Zamanı: {time.ctime()} ---")
    signals_df = scan_coins(coin_list)

    if not signals_df.empty:
        print(signals_df[['Coin', 'Close', 'Signal']])  # Konsolda göster
        save_to_excel(signals_df)  # Excel olarak kaydet
    else:
        print("Sinyal yok")

    time.sleep(300)  # 5 dakikada bir tarama



--- Tarama Zamanı: Mon Feb  3 18:53:58 2025 ---
           Coin          Close Signal
0       bitcoin  100065.000000   SELL
1      ethereum    2735.950000   SELL
2        solana     211.810000   SELL
3   binancecoin     599.890000   SELL
4      dogecoin       0.273093   SELL
5       cardano       0.783237   SELL
6     chainlink      20.810000   SELL
7   avalanche-2      27.470000   SELL
8       stellar       0.358267   SELL
9     shiba-inu       0.000016   SELL
10     litecoin     103.620000   SELL
11     polkadot       5.000000   SELL
12      uniswap       9.800000   SELL
13       monero     218.850000   SELL
📊 Excel dosyası kaydedildi: crypto_signals.xlsx
